In [ ]:
# cat hosts 

[amazon]

172.31.45.27 ansible_user="ec2-user" ansible_port=22 ansible_ssh_private_key_file="ansible.pem"

[ubuntu]

172.31.34.97 ansible_user="ubuntu" ansible_port=22 ansible_ssh_private_key_file="ansible.pem"

In [ ]:
$ ansible -i hosts amazon,ubuntu -b -f 1 -m ping 

The authenticity of host '172.31.45.27 (172.31.45.27)' can't be established.
ECDSA key fingerprint is SHA256:zFYZtKZIBGRSixVRsEUn9s+36atoGkLi/lyRNz3lOTg.
ECDSA key fingerprint is MD5:d6:9c:95:14:e9:01:a2:33:60:7d:05:ed:ca:5a:75:dc.
Are you sure you want to continue connecting (yes/no)? yes
[WARNING]: Platform linux on host 172.31.45.27 is using the discovered Python interpreter at /usr/bin/python, but future installation of another Python
interpreter could change this. See https://docs.ansible.com/ansible/2.9/reference_appendices/interpreter_discovery.html for more information.
172.31.45.27 | SUCCESS => {
    "ansible_facts": {
        "discovered_interpreter_python": "/usr/bin/python"
    }, 
    "changed": false, 
    "ping": "pong"
}
The authenticity of host '172.31.34.97 (172.31.34.97)' can't be established.
ECDSA key fingerprint is SHA256:loJZw5s0uQRMlrcXoWfF0XeRnS/ikyyGHCx1MGpV6xE.
ECDSA key fingerprint is MD5:d7:9b:07:8a:a3:b8:ef:83:98:41:85:5c:36:03:03:22.
Are you sure you want to continue connecting (yes/no)? yes
172.31.34.97 | SUCCESS => {
    "ansible_facts": {
        "discovered_interpreter_python": "/usr/bin/python3"
    }, 
    "changed": false, 
    "ping": "pong"
}


### by default if we have 12 servers in hosts first task run on the first 5 servers then the same task on next 5 servers and the same task run on the remaing 2 servers
### During the process if any task fail on any server the remaining task on that server will not run


In [ ]:
# vim amazon-ubuntu-apache.yml

---
- name: "installing apache on amazon & ubuntu"
  become: yes
  hosts: amazon,ubuntu
  tasks:
    
    - name: "Installing apache on amazon"
      yum:
        name: httpd
        state: present

    - name: "Creating index.html on amazon"
      copy:
        content: "<h1><center>Amazon-Index.html</h1></center>"
        dest: "/var/www/html/index.html"

    - name: "restarting/enabling amazon"
      service:
        name: httpd
        state: restarted
        enabled: yes

    - name: "Installing apache on ubuntu"
      apt:
        update_cache: yes
        name: apache2
        state: present
    
    - name: "Creating Index.html on ubuntu"
      copy:
        content: "<h1><center>ubuntu-Index.html</h1></center>"
        dest: "/var/www/html/index.html"
        
    - name: "Restarting/enabling apache on ubuntu"
      service:
        name: apache2
        state: restarted
        enabled: yes

In [ ]:
# ansible-playbook -i hosts amazon-ubuntu-apache.yml 

PLAY [installing apache on amazon & ubuntu] ***************************************************************************************************************

TASK [Gathering Facts] ************************************************************************************************************************************
ok: [172.31.34.97]
[WARNING]: Platform linux on host 172.31.45.27 is using the discovered Python interpreter at /usr/bin/python, but future installation of another Python
interpreter could change this. See https://docs.ansible.com/ansible/2.9/reference_appendices/interpreter_discovery.html for more information.
ok: [172.31.45.27]

TASK [Installing apache on amazon] ************************************************************************************************************************
fatal: [172.31.34.97]: FAILED! => {"ansible_facts": {"pkg_mgr": "apt"}, "cache_update_time": 1656427915, "cache_updated": false, "changed": false, "msg": "'/usr/bin/apt-get -y -o \"Dpkg::Options::=--force-confdef\" -o \"Dpkg::Options::=--force-confold\"      install 'httpd'' failed: E: Package 'httpd' has no installation candidate\n", "rc": 100, "stderr": "E: Package 'httpd' has no installation candidate\n", "stderr_lines": ["E: Package 'httpd' has no installation candidate"], "stdout": "Reading package lists...\nBuilding dependency tree...\nReading state information...\nPackage httpd is a virtual package provided by:\n  nginx-light 1.18.0-0ubuntu1.3\n  nginx-full 1.18.0-0ubuntu1.3\n  nginx-extras 1.18.0-0ubuntu1.3\n  lighttpd 1.4.55-1ubuntu1.20.04.1\n  nginx-core 1.18.0-0ubuntu1.3\n  apache2 2.4.41-4ubuntu3.12\n  yaws 2.0.7+dfsg-1\n  webfs 1.21+ds1-12\n  tntnet 2.2.1-4build1\n  ocsigenserver 2.16.0-2\n  mini-httpd 1.30-2\n  micro-httpd 20140814-2\n\n", "stdout_lines": ["Reading package lists...", "Building dependency tree...", "Reading state information...", "Package httpd is a virtual package provided by:", "  nginx-light 1.18.0-0ubuntu1.3", "  nginx-full 1.18.0-0ubuntu1.3", "  nginx-extras 1.18.0-0ubuntu1.3", "  lighttpd 1.4.55-1ubuntu1.20.04.1", "  nginx-core 1.18.0-0ubuntu1.3", "  apache2 2.4.41-4ubuntu3.12", "  yaws 2.0.7+dfsg-1", "  webfs 1.21+ds1-12", "  tntnet 2.2.1-4build1", "  ocsigenserver 2.16.0-2", "  mini-httpd 1.30-2", "  micro-httpd 20140814-2", ""]}
changed: [172.31.45.27]

TASK [Creating index.html on amazon] **********************************************************************************************************************
changed: [172.31.45.27]

TASK [restarting/enabling amazon] *************************************************************************************************************************
changed: [172.31.45.27]

TASK [Installing apache on ubuntu] ************************************************************************************************************************
[WARNING]: Updating cache and auto-installing missing dependency: python-apt
fatal: [172.31.45.27]: FAILED! => {"changed": false, "cmd": "apt-get update", "msg": "[Errno 2] No such file or directory", "rc": 2}

PLAY RECAP ************************************************************************************************************************************************
172.31.34.97               : ok=1    changed=0    unreachable=0    failed=1    skipped=0    rescued=0    ignored=0   
172.31.45.27               : ok=4    changed=3    unreachable=0    failed=1    skipped=0    rescued=0    ignored=0 

## Facts Gathering

### setup module

In [ ]:
$ ansible -i hosts amazon -m setup


[WARNING]: Platform linux on host 172.31.45.27 is using the discovered Python interpreter at /usr/bin/python, but future installation of another Python
interpreter could change this. See https://docs.ansible.com/ansible/2.9/reference_appendices/interpreter_discovery.html for more information.
172.31.45.27 | SUCCESS => {
    "ansible_facts": {
        "ansible_all_ipv4_addresses": [
            "172.31.45.27"
        ], 
        "ansible_all_ipv6_addresses": [
            "fe80::e4:90ff:fec8:b1d6"
        ], 
        "ansible_apparmor": {
            "status": "disabled"
        }, 
        "ansible_architecture": "x86_64", 
        "ansible_bios_date": "08/24/2006", 
        "ansible_bios_version": "4.11.amazon", 
        "ansible_cmdline": {
            "BOOT_IMAGE": "/boot/vmlinuz-5.10.118-111.515.amzn2.x86_64", 
            "biosdevname": "0", 
            "console": "ttyS0,115200n8", 
            "net.ifnames": "0", 
            "nvme_core.io_timeout": "4294967295", 
            "rd.emergency": "poweroff", 
            "rd.shell": "0", 
            "ro": true, 
            "root": "UUID=f33e7d3b-6786-4f40-875a-90f0f7e6b9a8"
        }, 


In [ ]:
# cat facts.yml 
---
- name: "printing hostname"
  hosts: amazon,ubuntu
  become: yes
  tasks:
    - name: "printing remote server hostname"
      debug:
        var: ansible_fqdn


In [ ]:
# ansible-playbook -i hosts facts.yml 

PLAY [printing hostname] **********************************************************************************************************************************

TASK [Gathering Facts] ************************************************************************************************************************************
ok: [172.31.34.97]
[WARNING]: Platform linux on host 172.31.45.27 is using the discovered Python interpreter at /usr/bin/python, but future installation of another Python
interpreter could change this. See https://docs.ansible.com/ansible/2.9/reference_appendices/interpreter_discovery.html for more information.
ok: [172.31.45.27]

TASK [printing remote server hostname] ********************************************************************************************************************
ok: [172.31.45.27] => {
    "ansible_fqdn": "ip-172-31-45-27.ap-south-1.compute.internal"
}
ok: [172.31.34.97] => {
    "ansible_fqdn": "ip-172-31-34-97.ap-south-1.compute.internal"
}

PLAY RECAP ************************************************************************************************************************************************
172.31.34.97               : ok=2    changed=0    unreachable=0    failed=0    skipped=0    rescued=0    ignored=0   
172.31.45.27               : ok=2    changed=0    unreachable=0    failed=0    skipped=0    rescued=0    ignored=0   

#### ansible_distribution & ansible_os_family

In [ ]:
# vim cat facts.yml 


---
- name: "printing distribution and os family"
  hosts: all 
  become: yes
  tasks:
    - name: "distribution and family of ubuntu and amazon"
      debug:
        msg: "My distribution is {{ ansible_distribution }} and Os Family {{ ansible_os_family }}"


In [ ]:
 ansible-playbook -i hosts facts.yml 

PLAY [printing distribution and os family] ****************************************************************************************************************

TASK [Gathering Facts] ************************************************************************************************************************************
ok: [172.31.34.97]
ok: [172.31.45.27]

TASK [distribution and family of ubuntu and amazon] *******************************************************************************************************
ok: [172.31.45.27] => {
    "msg": "My distribution is Amazon and Os Family RedHat"
}
ok: [172.31.34.97] => {
    "msg": "My distribution is Ubuntu and Os Family Debian"
}

PLAY RECAP ************************************************************************************************************************************************
172.31.34.97               : ok=2    changed=0    unreachable=0    failed=0    skipped=0    rescued=0    ignored=0   
172.31.45.27               : ok=2    changed=0    unreachable=0    failed=0    skipped=0    rescued=0    ignored=0 

#### memory facts gathering from mapped variable

In [ ]:
$ ansible -i hosts amazon -m setup|grep -A 15 -i ansible_memory_mb


        "ansible_memory_mb": {
            "nocache": {
                "free": 760, 
                "used": 205
            }, 
            "real": {
                "free": 226, 
                "total": 965, 
                "used": 739
            }, 
            "swap": {
                "cached": 0, 
                "free": 0, 
                "total": 0, 
                "used": 0
            }


In [ ]:
# cat facts.yml 

---
- name: "printing memory"
  hosts: all 
  become: yes
  tasks:
    - name: "memory of ubuntu and amazon"
      debug:
        msg: " Total Memory {{ ansible_memory_mb.real.total }} /Used memory {{ ansible_memory_mb.real.used }} /Free memory {{ ansible_memory_mb.real.free }} "


In [ ]:
$ ansible-playbook -i hosts facts.yml 


PLAY [printing memory] ************************************************************************************************************************************

TASK [Gathering Facts] ************************************************************************************************************************************
ok: [172.31.34.97]
ok: [172.31.45.27]

TASK [memory of ubuntu and amazon] ************************************************************************************************************************
ok: [172.31.45.27] => {
    "msg": " Total Memory 965 /Used memory 743 /Free memory 222 "
}
ok: [172.31.34.97] => {
    "msg": " Total Memory 967 /Used memory 869 /Free memory 98 "
}

PLAY RECAP ************************************************************************************************************************************************
172.31.34.97               : ok=2    changed=0    unreachable=0    failed=0    skipped=0    rescued=0    ignored=0   
172.31.45.27               : ok=2    changed=0    unreachable=0    failed=0    skipped=0    rescued=0    ignored=0

## Apache installtion on amazon and ubuntu based on condition

## skipping task for servers based on amazon-redhat and ubuntu-debian

In [ ]:
# cat amazon-ubuntu-apache.yml 


---
- name: "installing apache on amazon & ubuntu"
  become: yes
  hosts: amazon,ubuntu
  tasks:
    
    - name: "Installing apache on amazon"
      when: ansible_distribution == "Amazon" and  ansible_os_family == "RedHat"
      yum:
        name: httpd
        state: present

    - name: "Creating index.html on amazon"
      when: ansible_distribution == "Amazon" and  ansible_os_family == "RedHat"
      copy:
        content: " <h1><center>{{ ansible_distribution }}-{{ ansible_os_family }} - Index.html</h1></center> "
        dest: "/var/www/html/index.html"

    - name: "restarting/enabling amazon"
      when: ansible_distribution == "Amazon" and  ansible_os_family == "RedHat"
      service:
        name: httpd
        state: restarted
        enabled: yes

    - name: "Installing apache on ubuntu"
      when: ansible_distribution == "Ubuntu" and  ansible_os_family == "Debian"
      apt:
        update_cache: yes
        name: apache2
        state: present
    
    - name: "Creating Index.html on ubuntu"
      when: ansible_distribution == "Ubuntu" and  ansible_os_family == "Debian"
      copy:
        content: "<h1><center>{{ ansible_distribution }}-{{ ansible_os_family }} - Index.html</h1></center>"
        dest: "/var/www/html/index.html"
        
    - name: "Restarting/enabling apache on ubuntu"
      when: ansible_distribution == "Ubuntu" and  ansible_os_family == "Debian"
      service:
        name: apache2
        state: restarted
        enabled: yes


In [ ]:
$ ansible-playbook -i hosts amazon-ubuntu-apache.yml 


PLAY [installing apache on amazon & ubuntu] ***************************************************************************************************************

TASK [Gathering Facts] ************************************************************************************************************************************
ok: [172.31.34.97]
ok: [172.31.45.27]

TASK [Installing apache on amazon] ************************************************************************************************************************
skipping: [172.31.34.97]
ok: [172.31.45.27]

TASK [Creating index.html on amazon] **********************************************************************************************************************
skipping: [172.31.34.97]
changed: [172.31.45.27]

TASK [restarting/enabling amazon] *************************************************************************************************************************
skipping: [172.31.34.97]
changed: [172.31.45.27]

TASK [Installing apache on ubuntu] ************************************************************************************************************************
skipping: [172.31.45.27]
changed: [172.31.34.97]

TASK [Creating Index.html on ubuntu] **********************************************************************************************************************
skipping: [172.31.45.27]
changed: [172.31.34.97]

TASK [Restarting/enabling apache on ubuntu] ***************************************************************************************************************
skipping: [172.31.45.27]
changed: [172.31.34.97]

PLAY RECAP ************************************************************************************************************************************************
172.31.34.97               : ok=4    changed=3    unreachable=0    failed=0    skipped=3    rescued=0    ignored=0   
172.31.45.27               : ok=4    changed=2    unreachable=0    failed=0    skipped=3    rescued=0    ignored=0

## variables

In [ ]:
# cat /etc/apache2/envvars|grep -v "#"

unset HOME

else
	SUFFIX=
fi

export APACHE_RUN_USER=www-data
export APACHE_RUN_GROUP=www-data
export APACHE_PID_FILE=/var/run/apache2$SUFFIX/apache2.pid
export APACHE_RUN_DIR=/var/run/apache2$SUFFIX
export APACHE_LOCK_DIR=/var/lock/apache2$SUFFIX
export APACHE_LOG_DIR=/var/log/apache2$SUFFIX

export LANG=C

export LANG

# cat /etc/httpd/conf/httpd.conf |grep -v "#"

ServerRoot "/etc/httpd"

Listen 80

Include conf.modules.d/*.conf

User apache
Group apache


In [ ]:
# cat amazon-ubuntu-apache.yml 


---
- name: "installing apache on amazon & ubuntu"
  become: yes
  hosts: amazon,ubuntu
  vars:
    ubuntu_apache_user: "www-data"
    ubuntu_apache_group: "www-data"
    amazon_apache_user: "apache"
    amazon_apache_group: "apache"

  tasks:
    
    - name: "Installing apache on amazon"
      when: ansible_distribution == "Amazon" and  ansible_os_family == "RedHat"
      yum:
        name: httpd
        state: present

    - name: "Creating index.html on amazon"
      when: ansible_distribution == "Amazon" and  ansible_os_family == "RedHat"
      copy:
        content: " <h1><center>{{ ansible_distribution }}-{{ ansible_os_family }} - Index.html</h1></center> "
        dest: "/var/www/html/index.html"
        owner: "{{ amazon_apache_user }}"
        group: "{{ amazon_apache_group }}"

    - name: "restarting/enabling amazon"
      when: ansible_distribution == "Amazon" and  ansible_os_family == "RedHat"
      service:
        name: httpd
        state: restarted
        enabled: yes

    - name: "Installing apache on ubuntu"
      when: ansible_distribution == "Ubuntu" and  ansible_os_family == "Debian"
      apt:
        update_cache: yes
        name: apache2
        state: present
    
    - name: "Creating Index.html on ubuntu"
      when: ansible_distribution == "Ubuntu" and  ansible_os_family == "Debian"
      copy:
        content: "<h1><center>{{ ansible_distribution }}-{{ ansible_os_family }} - Index.html</h1></center>"
        dest: "/var/www/html/index.html"
        owner: "{{ ubuntu_apache_user }}"
        group: "{{ ubuntu_apache_group }}"
        
    - name: "Restarting/enabling apache on ubuntu"
      when: ansible_distribution == "Ubuntu" and  ansible_os_family == "Debian"
      service:
        name: apache2
        state: restarted
        enabled: yes


In [ ]:
$ ansible-playbook -i hosts amazon-ubuntu-apache.yml 


PLAY [installing apache on amazon & ubuntu] ***************************************************************************************************************

TASK [Gathering Facts] ************************************************************************************************************************************
ok: [172.31.45.27]
ok: [172.31.34.97]

TASK [Installing apache on amazon] ************************************************************************************************************************
skipping: [172.31.34.97]
ok: [172.31.45.27]

TASK [Creating index.html on amazon] **********************************************************************************************************************
skipping: [172.31.34.97]
changed: [172.31.45.27]

TASK [restarting/enabling amazon] *************************************************************************************************************************
skipping: [172.31.34.97]
changed: [172.31.45.27]

TASK [Installing apache on ubuntu] ************************************************************************************************************************
skipping: [172.31.45.27]
ok: [172.31.34.97]

TASK [Creating Index.html on ubuntu] **********************************************************************************************************************
skipping: [172.31.45.27]
changed: [172.31.34.97]

TASK [Restarting/enabling apache on ubuntu] ***************************************************************************************************************
skipping: [172.31.45.27]
changed: [172.31.34.97]

PLAY RECAP ************************************************************************************************************************************************
172.31.34.97               : ok=4    changed=2    unreachable=0    failed=0    skipped=3    rescued=0    ignored=0   
172.31.45.27               : ok=4    changed=2    unreachable=0    failed=0    skipped=3    rescued=0    ignored=0 

### Task level variables

In [ ]:
# cat amazon-ubuntu-apache.yml 

---
- name: "installing apache on amazon & ubuntu"
  become: yes
  hosts: amazon,ubuntu

  tasks:
    
    - name: "Installing apache on amazon"
      when: ansible_distribution == "Amazon" and  ansible_os_family == "RedHat"
      yum:
        name: httpd
        state: present

    - name: "Creating index.html on amazon"
      when: ansible_distribution == "Amazon" and  ansible_os_family == "RedHat"
      copy:
        content: " <h1><center>{{ ansible_distribution }}-{{ ansible_os_family }} - Index.html</h1></center> "
        dest: "/var/www/html/index.html"
        owner: "{{ apache_user }}"
        group: "{{ apache_group }}"
      vars:
        apache_user: "apache"
        apache_group: "apache"

    - name: "restarting/enabling amazon"
      when: ansible_distribution == "Amazon" and  ansible_os_family == "RedHat"
      service:
        name: httpd
        state: restarted
        enabled: yes

    - name: "Installing apache on ubuntu"
      when: ansible_distribution == "Ubuntu" and  ansible_os_family == "Debian"
      apt:
        update_cache: yes
        name: apache2
        state: present
    
    - name: "Creating Index.html on ubuntu"
      when: ansible_distribution == "Ubuntu" and  ansible_os_family == "Debian"
      copy:
        content: "<h1><center>{{ ansible_distribution }}-{{ ansible_os_family }} - Index.html</h1></center>"
        dest: "/var/www/html/index.html"
        owner: "{{ apache_user }}"
        group: "{{ apache_group }}"
      vars:
        apache_user: "www-data"
        apache_group: "www-data"
        
    - name: "Restarting/enabling apache on ubuntu"
      when: ansible_distribution == "Ubuntu" and  ansible_os_family == "Debian"
      service:
        name: apache2
        state: restarted
        enabled: yes


In [ ]:
$ ansible-playbook -i hosts amazon-ubuntu-apache.yml 


PLAY [installing apache on amazon & ubuntu] ***************************************************************************************************************

TASK [Gathering Facts] ************************************************************************************************************************************
ok: [172.31.34.97]
ok: [172.31.45.27]

TASK [Installing apache on amazon] ************************************************************************************************************************
skipping: [172.31.34.97]
ok: [172.31.45.27]

TASK [Creating index.html on amazon] **********************************************************************************************************************
skipping: [172.31.34.97]
ok: [172.31.45.27]

TASK [restarting/enabling amazon] *************************************************************************************************************************
skipping: [172.31.34.97]
changed: [172.31.45.27]

TASK [Installing apache on ubuntu] ************************************************************************************************************************
skipping: [172.31.45.27]
ok: [172.31.34.97]

TASK [Creating Index.html on ubuntu] **********************************************************************************************************************
skipping: [172.31.45.27]
ok: [172.31.34.97]

TASK [Restarting/enabling apache on ubuntu] ***************************************************************************************************************
skipping: [172.31.45.27]
changed: [172.31.34.97]

PLAY RECAP ************************************************************************************************************************************************
172.31.34.97               : ok=4    changed=1    unreachable=0    failed=0    skipped=3    rescued=0    ignored=0   
172.31.45.27               : ok=4    changed=1    unreachable=0    failed=0    skipped=3    rescued=0    ignored=0 

## set_fact module

In [ ]:
# dynamic variable assigning

In [ ]:
$ cat amazon-ubuntu-apache.yml 


---
- name: "installing apache on amazon & ubuntu"
  become: yes
  hosts: amazon,ubuntu

  tasks:

    - name: "amazon setting variables"
      when: ansible_distribution == "Amazon"
      set_fact:
        apache_user: "apache"
        apache_group: "apache"

    - name: "Ubuntu setting variable"
      when: ansible_distribution == "Ubuntu"
      set_fact:
        apache_user: "www-data"
        apache_group: "www-data"
    
    - name: "Installing apache on amazon"
      when: ansible_distribution == "Amazon" and  ansible_os_family == "RedHat"
      yum:
        name: httpd
        state: present

    - name: "Creating index.html on amazon"
      when: ansible_distribution == "Amazon" and  ansible_os_family == "RedHat"
      copy:
        content: " <h1><center>{{ ansible_distribution }}-{{ ansible_os_family }} - Index.html</h1></center> "
        dest: "/var/www/html/index.html"
        owner: "{{ apache_user }}"
        group: "{{ apache_group }}"


    - name: "restarting/enabling amazon"
      when: ansible_distribution == "Amazon" and  ansible_os_family == "RedHat"
      service:
        name: httpd
        state: restarted
        enabled: yes

    - name: "Installing apache on ubuntu"
      when: ansible_distribution == "Ubuntu" and  ansible_os_family == "Debian"
      apt:
        update_cache: yes
        name: apache2
        state: present
    
    - name: "Creating Index.html on ubuntu"
      when: ansible_distribution == "Ubuntu" and  ansible_os_family == "Debian"
      copy:
        content: "<h1><center>{{ ansible_distribution }}-{{ ansible_os_family }} - Index.html</h1></center>"
        dest: "/var/www/html/index.html"
        owner: "{{ apache_user }}"
        group: "{{ apache_group }}"
        
    - name: "Restarting/enabling apache on ubuntu"
      when: ansible_distribution == "Ubuntu" and  ansible_os_family == "Debian"
      service:
        name: apache2
        state: restarted
        enabled: yes


In [ ]:
$ ansible-playbook -i hosts amazon-ubuntu-apache.yml 



PLAY [installing apache on amazon & ubuntu] ***************************************************************************************************************

TASK [Gathering Facts] ************************************************************************************************************************************
ok: [172.31.34.97]
ok: [172.31.45.27]

TASK [amazon setting variables] ***************************************************************************************************************************
ok: [172.31.45.27]
skipping: [172.31.34.97]

TASK [Ubuntu setting variable] ****************************************************************************************************************************
skipping: [172.31.45.27]
ok: [172.31.34.97]

TASK [Installing apache on amazon] ************************************************************************************************************************
skipping: [172.31.34.97]
ok: [172.31.45.27]

TASK [Creating index.html on amazon] **********************************************************************************************************************
skipping: [172.31.34.97]
ok: [172.31.45.27]

TASK [restarting/enabling amazon] *************************************************************************************************************************
skipping: [172.31.34.97]
changed: [172.31.45.27]

TASK [Installing apache on ubuntu] ************************************************************************************************************************
skipping: [172.31.45.27]
ok: [172.31.34.97]

TASK [Creating Index.html on ubuntu] **********************************************************************************************************************
skipping: [172.31.45.27]
ok: [172.31.34.97]

TASK [Restarting/enabling apache on ubuntu] ***************************************************************************************************************
skipping: [172.31.45.27]
changed: [172.31.34.97]

PLAY RECAP ************************************************************************************************************************************************
172.31.34.97               : ok=5    changed=1    unreachable=0    failed=0    skipped=4    rescued=0    ignored=0   
172.31.45.27               : ok=5    changed=1    unreachable=0    failed=0    skipped=4    rescued=0    ignored=0

## mariadb-server installation

In [ ]:
#installing package
$ yum install mariadb-server -y

#restart/enable
$ sudo systemctl restart mariadb
$ sudo systemctl enable mariadb

In [ ]:
# cat mariadb.yml 
---
- name: "Installing mariadb server"
  hosts: amazon
  become: yes
  tasks:
    - name: "installing mariadb and pip packages"
      yum: 
        name: 
          - mariadb-server
          - pip 
        state: present

    - name: "Restarting/enabling mariadb server"
      service:
        name: mariadb
        state: restarted
        enabled: yes


In [ ]:
# ansible-playbook -i hosts mariadb.yml 

PLAY [Installing mariadb server] **************************************************************************************************************************

TASK [Gathering Facts] ************************************************************************************************************************************
The authenticity of host '172.31.37.108 (172.31.37.108)' can't be established.
ECDSA key fingerprint is SHA256:V3s9FTgtZPVnIHs/udR1SnYjJOYpw07j9vSOStoa76w.
ECDSA key fingerprint is MD5:06:1c:39:a7:37:c5:73:89:d0:2e:c0:63:e4:1c:a4:95.
Are you sure you want to continue connecting (yes/no)? yes
[WARNING]: Platform linux on host 172.31.37.108 is using the discovered Python interpreter at /usr/bin/python, but future installation of another Python
interpreter could change this. See https://docs.ansible.com/ansible/2.9/reference_appendices/interpreter_discovery.html for more information.
ok: [172.31.37.108]

TASK [installing mariadb and pip packages] ****************************************************************************************************************
changed: [172.31.37.108]

TASK [Restarting/enabling mariadb server] *****************************************************************************************************************
changed: [172.31.37.108]

PLAY RECAP ************************************************************************************************************************************************
172.31.37.108              : ok=3    changed=2    unreachable=0    failed=0    skipped=0    rescued=0    ignored=0 

In [ ]:

#login to mariadb with null password and reset root password for all hosts

$ mysql -u root
Welcome to the MariaDB monitor.  Commands end with ; or \g.
Your MariaDB connection id is 2
Server version: 5.5.68-MariaDB MariaDB Server

Copyright (c) 2000, 2018, Oracle, MariaDB Corporation Ab and others.

Type 'help;' or '\h' for help. Type '\c' to clear the current input statement.

MariaDB [(none)]> 

MariaDB [(none)]> select user,password,host from mysql.user;
+------+----------+---------------------------------------------+
| user | password | host                                        |
+------+----------+---------------------------------------------+
| root |          | localhost                                   |
| root |          | ip-172-31-45-27.ap-south-1.compute.internal |
| root |          | 127.0.0.1                                   |
| root |          | ::1                                         |
|      |          | localhost                                   |
|      |          | ip-172-31-45-27.ap-south-1.compute.internal |
+------+----------+---------------------------------------------+
6 rows in set (0.00 sec)


#setting root password

MariaDB [(none)]> update mysql.user set password=password('mysqlroot@123') where user='root';
Query OK, 4 rows affected (0.00 sec)
Rows matched: 4  Changed: 4  Warnings: 0

MariaDB [(none)]> flush privileges;
Query OK, 0 rows affected (0.00 sec)

MariaDB [(none)]> exit
Bye




In [ ]:
# cat mariadb.yml 
---
- name: "Installing mariadb server"
  hosts: amazon
  become: yes
  tasks:
    - name: "installing mariadb and pip packages"
      yum: 
        name: 
          - mariadb-server
          - pip 
        state: present

    - name: "Restarting/enabling mariadb server"
      service:
        name: mariadb
        state: restarted
        enabled: yes    

    - name: "Resetting root password"
      mysql_user:
        login_user: "root"
        login_password: ""
        name: "root"
        password: "mysqlroot@123*"
        host_all: yes


#### error

In [ ]:
# ansible-playbook -i hosts mariadb.yml 

PLAY [Installing mariadb server] **************************************************************************************************************************

TASK [Gathering Facts] ************************************************************************************************************************************
[WARNING]: Platform linux on host 172.31.37.108 is using the discovered Python interpreter at /usr/bin/python, but future installation of another Python
interpreter could change this. See https://docs.ansible.com/ansible/2.9/reference_appendices/interpreter_discovery.html for more information.
ok: [172.31.37.108]

TASK [installing mariadb and pip packages] ****************************************************************************************************************
ok: [172.31.37.108]

TASK [Restarting/enabling mariadb server] *****************************************************************************************************************
changed: [172.31.37.108]

TASK [Resetting root password] ****************************************************************************************************************************
[WARNING]: Module did not set no_log for update_password
fatal: [172.31.37.108]: FAILED! => {"changed": false, "msg": "The PyMySQL (Python 2.7 and Python 3.X) or MySQL-python (Python 2.X) module is required."}

PLAY RECAP ************************************************************************************************************************************************
172.31.37.108              : ok=3    changed=1    unreachable=0    failed=1    skipped=0    rescued=0    ignored=0 

#### fix

In [ ]:
# cat mariadb.yml 
---
- name: "Installing mariadb server"
  hosts: amazon
  become: yes
  tasks:
    - name: "installing mariadb and pip packages"
      yum: 
        name: 
          - mariadb-server
          - pip 
        state: present
    - name:
      pip:
        name: PyMySQL
        state: present

    - name: "Restarting/enabling mariadb server"
      service:
        name: mariadb
        state: restarted
        enabled: yes    

    - name: "Resetting root password"
      mysql_user:
        login_user: "root"
        login_password: ""
        name: "root"
        password: "mysqlroot@123*"
        host_all: yes


In [ ]:
# ansible-playbook -i hosts mariadb.yml 

PLAY [Installing mariadb server] **************************************************************************************************************************

TASK [Gathering Facts] ************************************************************************************************************************************
[WARNING]: Platform linux on host 172.31.37.108 is using the discovered Python interpreter at /usr/bin/python, but future installation of another Python
interpreter could change this. See https://docs.ansible.com/ansible/2.9/reference_appendices/interpreter_discovery.html for more information.
ok: [172.31.37.108]

TASK [installing mariadb and pip packages] ****************************************************************************************************************
ok: [172.31.37.108]

TASK [pip] ************************************************************************************************************************************************
changed: [172.31.37.108]

TASK [Restarting/enabling mariadb server] *****************************************************************************************************************
changed: [172.31.37.108]

TASK [Resetting root password] ****************************************************************************************************************************
[WARNING]: Module did not set no_log for update_password
changed: [172.31.37.108]

PLAY RECAP ************************************************************************************************************************************************
172.31.37.108              : ok=5    changed=3    unreachable=0    failed=0    skipped=0    rescued=0    ignored=0 

In [ ]:
#on client
MariaDB [(none)]> select user,password,host from mysql.user;
+------+-------------------------------------------+----------------------------------------------+
| user | password                                  | host                                         |
+------+-------------------------------------------+----------------------------------------------+
| root | *864E1F7E2334EF528CD1BABC82AE54A541A83236 | localhost                                    |
| root | *864E1F7E2334EF528CD1BABC82AE54A541A83236 | ip-172-31-37-108.ap-south-1.compute.internal |
| root | *864E1F7E2334EF528CD1BABC82AE54A541A83236 | 127.0.0.1                                    |
| root | *864E1F7E2334EF528CD1BABC82AE54A541A83236 | ::1                                          |
|      |                                           | localhost                                    |
|      |                                           | ip-172-31-37-108.ap-south-1.compute.internal |
+------+-------------------------------------------+----------------------------------------------+


#### task: removing anonymous users

In [ ]:
MariaDB [(none)]> delete from mysql.user where user='' and password='';
Query OK, 2 rows affected (0.00 sec)

MariaDB [(none)]> select user,password,host from mysql.user;
+------+----------+---------------------------------------------+
| user | password | host                                        |
+------+----------+---------------------------------------------+
| root |          | localhost                                   |
| root |          | ip-172-31-36-60.ap-south-1.compute.internal |
| root |          | 127.0.0.1                                   |
| root |          | ::1                                         |
+------+----------+---------------------------------------------+
4 rows in set (0.00 sec)


In [ ]:
# cat mariadb.yml 
---
- name: "Installing mariadb server"
  hosts: amazon
  become: yes
  tasks:
    - name: "installing mariadb and pip packages"
      yum: 
        name: 
          - mariadb-server
          - pip 
        state: present
    - name:
      pip:
        name: PyMySQL
        state: present

    - name: "Restarting/enabling mariadb server"
      service:
        name: mariadb
        state: restarted
        enabled: yes    

    - name: "Resetting root password"
      ignore_errors: yes
      mysql_user:
        login_user: "root"
        login_password: ""
        name: "root"
        password: "mysqlroot@123*"
        host_all: yes

    - name: "removing anonymous users"
      mysql_user:
        login_user: "root"
        login_password: "mysqlroot@123*"
        name: ""
        host_all: yes
        state: absent


In [ ]:
# ansible-playbook -i hosts mariadb.yml 

PLAY [Installing mariadb server] **************************************************************************************************************************

TASK [Gathering Facts] ************************************************************************************************************************************
[WARNING]: Platform linux on host 172.31.37.108 is using the discovered Python interpreter at /usr/bin/python, but future installation of another Python
interpreter could change this. See https://docs.ansible.com/ansible/2.9/reference_appendices/interpreter_discovery.html for more information.
ok: [172.31.37.108]

TASK [installing mariadb and pip packages] ****************************************************************************************************************
ok: [172.31.37.108]

TASK [pip] ************************************************************************************************************************************************
ok: [172.31.37.108]

TASK [Restarting/enabling mariadb server] *****************************************************************************************************************
changed: [172.31.37.108]

TASK [Resetting root password] ****************************************************************************************************************************
[WARNING]: Module did not set no_log for update_password
fatal: [172.31.37.108]: FAILED! => {"changed": false, "msg": "unable to connect to database, check login_user and login_password are correct or /root/.my.cnf has the credentials. Exception message: (1045, u\"Access denied for user 'root'@'localhost' (using password: NO)\")"}
...ignoring

TASK [removing anonymous users] ***************************************************************************************************************************
changed: [172.31.37.108]

PLAY RECAP ************************************************************************************************************************************************
172.31.37.108              : ok=6    changed=2    unreachable=0    failed=0    skipped=0    rescued=0    ignored=1  

In [ ]:
#on client machine

MariaDB [(none)]> select user,password,host from mysql.user;
+------+-------------------------------------------+----------------------------------------------+
| user | password                                  | host                                         |
+------+-------------------------------------------+----------------------------------------------+
| root | *864E1F7E2334EF528CD1BABC82AE54A541A83236 | localhost                                    |
| root | *864E1F7E2334EF528CD1BABC82AE54A541A83236 | ip-172-31-37-108.ap-south-1.compute.internal |
| root | *864E1F7E2334EF528CD1BABC82AE54A541A83236 | 127.0.0.1                                    |
| root | *864E1F7E2334EF528CD1BABC82AE54A541A83236 | ::1                                          |
+------+-------------------------------------------+----------------------------------------------+
4 rows in set (0.00 sec)
